[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/modelagem_2.ipynb)

# Modelagem e Gestão de Processos


**Prof. Diogo Ferreira de Lima Silva (TEP-UFF)**

Tema da aula - Modelagem de Processos com a biblioteca SimPy

In [ ]:
#!pip install simpy
import simpy

In [ ]:
import random
random.seed(10)

# Monitorando o Processo

Ao analisar um processo, um gestor pode estar interessado em algumas métricas, tais como:
- Tempo de ciclo médio;
- Tempo de espera médio em fila;
- Uso dos recursos
- Trabalho em processo médio;

Para fazer tais cálculos, precisamos colher algumas informações durante a simulação.

### Vamos incrementar nosso código para guardar o tempo gasto na atividade C

In [ ]:
#criando uma lista vazia
tempo_gasto_C = []

tempo_esperando_C =[]

In [ ]:
def salao (environment, nome, colaborador_A, colaborador_B, colaborador_C):
    #print (f"{nome} chega no estabelecimento em {env.now}")
    # Serviço A
    with colaborador_A.request() as req_A:
        yield req_A # o cliente deve esperar um recurso do tipo colaborador_A
        #print (f"{nome} inicia A em {env.now}") # imprima na tela o tempo de início
        yield environment.timeout(3) # O serviço demora exatamente 3 minutos.
        #print (f"{nome} finaliza A em {env.now}")
    
    # Serviço B
    with colaborador_B.request() as req_B:
        yield req_B
        #print (f"{nome} inicia B em {env.now}")
        yield environment.timeout(8) # O serviço B demora exatamente 8 minutos.
        #print (f"{nome} finaliza B em {env.now}")

        acaba_B = environment.now

    # Serviço C
    with colaborador_C.request() as req_C:
        yield req_C
        #print (f"{nome} inicia C em {env.now}")
        
        p_inicio = environment.now
        yield environment.timeout(random.expovariate(1/5)) # O tempo de serviço em C segue uma exponencial de média 5.
        p_fim = environment.now

        tempo_gasto_C.append(p_fim - p_inicio)
        tempo_esperando_C.append(p_inicio - acaba_B)
        
    #print (f"{nome} sai do estabelecimento em {env.now}")
    print (f"{nome} esperou {p_inicio - acaba_B} na fila da atividade C")
    print (f"{nome} gastou {p_fim - p_inicio} na atividade C")

In [ ]:
def chegadas (env):
    i=1 # número que será usado na contagem e nomeação dos clientes
    
    # Enquanto a simulação ocorrer
    while True:
        # gere um número de nossa distribuição 
        yield env.timeout(random.expovariate(1/5))

        # Um cliente entra no processo! Seu nome será "Cliente i"
        env.process(salao (env, 'Cliente %d' % i, colab_A, colab_B, colab_C))
        
        i+=1 # i = i+1


In [ ]:
env = simpy.Environment()
colab_A = simpy.Resource(env, capacity=1)
colab_B = simpy.Resource(env, capacity=1)
colab_C = simpy.Resource(env, capacity=1)
env.process(chegadas(env))
env.run(until = 6000)

In [ ]:
import numpy as np

print(f"Em média, um cliente o serviço C dura {np.mean(tempo_gasto_C)} minutos")
print(f"Em média, um cliente espera {np.mean(tempo_esperando_C)} minutos pelo serviço C")
